In [ ]:
from math import sin, cos
import numpy as np
import numpy.linalg as la
import sympy as sp

In [ ]:
# HW 16.2


In [ ]:
# HW 16.4
f = lambda x: cos(x)
df = lambda x: -sin(x)
d2f = lambda x: -cos(x)
x0 = 0.25
x1 = x0 - df(x0)/d2f(x0)
x1

In [ ]:
def gradient(formula, symbols, values=None):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae or numerical gradient
    if values specified
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
        
    if values == None:
        return gradient

    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at

def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())

def hessian(gradient, symbols, values=None):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    or numerical Hessian if values are specified
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
            
    if values == None:
        return hessian
    
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            hessian_at[i][j] = subs_all(hessian_at[i][j], symbols, [float(v) for v in values])
#             for k in range(len(symbols)):
#                 hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
#             hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at

def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    print(neg_gradient_at)
    hes_at = np.array(hessian(grad, symbols, values=x_prev))
    print(hes_at)
    print(np.array(x_prev) + la.solve(hes_at, neg_gradient_at))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at)

def newton_nd_optimization_crude(f_str, s_str, start, tolerance, actual_solution):
    '''
    A crude version of Newton ND Optimization algorithm
    Maybe you can help out implementing an analytic solution
    Returning the numerical solution as well as the number of
    iterations it took
    '''
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    while (la.norm(curr - actual_solution, 2) > tolerance):
        curr = newton_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    return curr, iterations

In [ ]:
# HW 16.10
newton_nd_optimization_crude(f_str = '20*x**2-6*x+6+20*y**2-x*y', 
                             s_str = 'x y', 
                             start = [1, 2], 
                             tolerance = 1e-10,
                             actual_solution = [240/1599, 6/1599])

In [ ]:
# HW 16.11
newton_nd_optimization_crude(f_str = '3*x**2+2*x*y+6*y**2', 
                             s_str = 'x y', 
                             start = [9, 6], 
                             tolerance = 1,
                             actual_solution = [0, 0])

In [ ]:
# HW 16.12
newton_nd_optimization_crude(f_str = '2*x**2+11*x*y+2*y**2+6*E**(14*x*y)+9*sin(y)**2+9*cos(x*y)', 
                             s_str = 'x y', 
                             start = [0, 1], 
                             tolerance = 1e-10,
                             actual_solution = [0, 0])

In [ ]:
# HW 16.15

from math import sin, cos
import numpy as np
import numpy.linalg as la
import sympy as sp
import scipy.optimize as opt
import matplotlib.pyplot as plt

def f(alpha, r, s):
    r += s*alpha
    x, y = r
    return 3 +((x**2)/8) + ((y**2)/8) - np.sin(x)*np.cos((2**-0.5)*y)

def sd_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    alpha = opt.minimize_scalar(f, args=(x_prev, neg_gradient_at)).x
    return np.array(x_prev) + alpha * neg_gradient_at, la.norm(neg_gradient_at, 2)

def gradient(formula, symbols, values=None):
    '''
    Given a SymPy formula and variables
    Find its analytic gradient without substituion
    as a list of SymPy formulae or numerical gradient
    if values specified
    '''
    size = len(symbols)
    gradient = []
    for i in range(size):
        gradient.append(formula.diff(symbols[i]))
        
    if values == None:
        return gradient

    # Make sure you don't mess up the analytical gradient
    g_copy = gradient.copy()
    gradient_at = []
    for i in range(len(g_copy)):
        for j in range(len(symbols)):
            g_copy[i] = g_copy[i].subs(symbols[j], values[j])
        gradient_at.append(float(g_copy[i].evalf()))
    return gradient_at

def subs_all(formula, variables, values):
    '''
    You know what, it's getting to the point where this function
    is necessary
    '''
    result = formula
    for i in range(len(values)):
        result = result.subs(variables[i], values[i])
    return float(result.evalf())

def hessian(gradient, symbols, values=None):
    '''
    Given an analytic gradient and variables
    Calculate its analytic Hessian
    or numerical Hessian if values are specified
    '''
    size = len(symbols)
    hessian = []
    for i in range(size):
        hessian.append([0]*size)
    for i in range(size):
        for j in range(size):
            hessian[i][j] = gradient[i].diff(symbols[j])
            
    if values == None:
        return hessian
    
    hessian_at = hessian.copy()
    size = len(hessian)
    for i in range(size):
        for j in range(size):
            hessian_at[i][j] = subs_all(hessian_at[i][j], symbols, [float(v) for v in values])
#             for k in range(len(symbols)):
#                 hessian_at[i][j] = hessian_at[i][j].subs(symbols[k], values[k])
#             hessian_at[i][j] = float(hessian_at[i][j].evalf())
    return hessian_at

def newton_nd_optimization_crude_step(formula, symbols, x_prev):
    x_prev = list(x_prev)
    grad = gradient(formula, symbols)
    neg_gradient_at = -1 * np.array(gradient(formula, symbols, values=x_prev))
    hes_at = np.array(hessian(grad, symbols, values=x_prev))
    return np.array(x_prev) + la.solve(hes_at, neg_gradient_at), la.norm(neg_gradient_at, 2)

def newton_nd_optimization_crude(f_str, s_str, start, tolerance):
    '''
    A crude version of Newton ND Optimization algorithm
    Maybe you can help out implementing an analytic solution
    Returning the numerical solution as well as the number of
    iterations it took
    '''
    x_list = []
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    gradient = np.inf
    while (gradient > tolerance):
        x_list.append(curr)
        curr, gradient = newton_nd_optimization_crude_step(formula, symbols, curr)
        _, gradient = newton_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    x_list.append(curr)
    return curr, iterations, x_list

def sd_nd_optimization_crude(f_str, s_str, start, tolerance):
    formula = sp.sympify(f_str)
    symbols = sp.symbols(s_str)
    curr = np.copy(start)
    iterations = 0
    gradient = np.inf
    x_list = []
    while (gradient > tolerance):
        x_list.append(curr)
        curr, gradient = sd_nd_optimization_crude_step(formula, symbols, curr)
        iterations += 1
    x_list.append(curr)
    return curr, iterations, x_list


r_newton, iteration_count_newton, newton_list = newton_nd_optimization_crude(f_str = '3 +((x**2)/8) + ((y**2)/8) - sin(x)*cos((2**-0.5)*y)', 
                                                                s_str = 'x y', 
                                                                start = r_init, 
                                                                tolerance = stop)

r_sd, iteration_count_sd, sd_list = sd_nd_optimization_crude(f_str = '3 +((x**2)/8) + ((y**2)/8) - sin(x)*cos((2**-0.5)*y)', 
                                                    s_str = 'x y', 
                                                    start = r_init, 
                                                    tolerance = stop*10)


newton_plot = [np.log(la.norm(r-r_newton,2)) for r in newton_list]
sd_plot = [np.log(la.norm(r-r_sd,2)) for r in sd_list]

plt.plot(sd_plot, label='Steepest descent line')
plt.plot(newton_plot, label='newton')

plt.xlabel('Iteration')
plt.ylabel('log(Error)')
plt.legend()
plt.title('Error vs. Iteration')